In [1]:
# 產生QRcode 範例
import qrcode
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4
)
qr.add_data('https://bmed.tahsda.org.tw/bmedmgt/MedEngMgt/Assets/Details/BME0000019')   # 要轉換成 QRCode 的文字
qr.make(fit=True)          # 根據參數製作為 QRCode 物件

img = qr.make_image()      # 產生 QRCode 圖片
img.show()
img.save('qrcode_BME.png')    

In [3]:
# 正常設備名單
import pandas as pd
file_path = 'E:/Python Practice/QRcode/20240119設備清單.xlsx'
df_eqi = pd.read_excel(file_path, engine='openpyxl')

eqi_dict = df_eqi[['財產編號', '設備編號']].set_index('財產編號').to_dict()['設備編號']

In [4]:
# 篩選出財產編號相同的醫材

# print(len(eqi_dict)) # 7157
# print(len(df_eqi)) # 7171
duplicate_groups = df_eqi[df_eqi['財產編號'].duplicated(keep=False)].sort_values('財產編號')

for _, group in duplicate_groups.groupby('財產編號'):
    print(f"財產編號: {group['財產編號'].iloc[0]}")
    print(group[['財產編號', '設備編號']])
    print("="*30)

財產編號: 201812002301
              財產編號        設備編號
823   201812002301  BME0002315
5884  201812002301  PUR0016804
財產編號: 201812002302
              財產編號        設備編號
824   201812002302  BME0002316
5885  201812002302  PUR0016805
財產編號: 201812002303
              財產編號        設備編號
5886  201812002303  PUR0016806
825   201812002303  BME0002317
財產編號: 201902005400
              財產編號        設備編號
762   201902005400  BME0002251
5933  201902005400  PUR0016860
財產編號: 201903001500
              財產編號        設備編號
5948  201903001500  PUR0016885
751   201903001500  BME0002239
財產編號: 201911001400
              財產編號        設備編號
6005  201911001400  PUR0016987
834   201911001400  BME0002326
財產編號: 202003003000
              財產編號        設備編號
935   202003003000  BME0002428
6150  202003003000  PUR0017240
財產編號: 202011000800
              財產編號        設備編號
6131  202011000800  PUR0017204
1002  202011000800  BME0002503
財產編號: 202011001000
              財產編號        設備編號
6132  202011001000  PUR0017206
1258  202011001000  BME

函式撰寫

In [5]:
# QRcode 大量生成：設備資訊
import os
import requests
import qrcode
import pandas as pd

def check_and_generate_qrcode(base_url, prefix, stop_at, save_path,eqi_dict):
    current_number = 1

    while True:
        current_url = f'{base_url}/{prefix}{str(current_number).zfill(7)}'
        number = str(current_url.split("/")[-1])
        
        # 判斷是否達到停止條件
        if current_number > stop_at:
            print(f'已達到停止條件 {stop_at}，停止生成 QR code。')
            break

        # # 檢查網址是否存在
        # response = requests.get(current_url)
        # if response.status_code != 200 or response.status_code != 404:
        #     current_number += 1
        #     continue

        # 醫材狀態正常，生成QR code
        elif number in eqi_dict.values():
            qr = qrcode.QRCode(version=1,error_correction=qrcode.constants.ERROR_CORRECT_L,box_size=10,border=4)
            qr.add_data(current_url)
            qr.make(fit=True)

            # 存檔QRcode，財產編號作為檔名
            file_name = list(eqi_dict.keys())[list(eqi_dict.values()).index(number)]
            file_path = os.path.join(save_path, f'{file_name}_設備基本資料.png')
            img = qr.make_image()
            img.save(file_path)
            current_number += 1
            print(number)

        else:
            current_number += 1
            continue


實際測試

In [ ]:
base_url = 'https://bmed.tahsda.org.tw/bmedmgt/MedEngMgt/Assets/Details'
save_path = 'E:/Python Practice/QRcode/設備資料QRcode/'

check_and_generate_qrcode(base_url, 'PUR', 18400, save_path,eqi_dict)
check_and_generate_qrcode(base_url, 'BME', 2850, save_path,eqi_dict)